# Install dependencies

In [1]:
%%capture --no-stderr
%pip install -U --quiet 'crewai[tools]' aisuite

# Set environment variables

In [2]:
import getpass
import time
initial_time = time.time()

import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Create Crew

In [3]:
# Importing Crew related components
# Importing CrewAI Flow related components
# Importing CrewAI Tools
from crewai import Agent, Task, Crew
from crewai.flow.flow import Flow, listen, start
from crewai_tools import WebsiteSearchTool

# Importing AI Suite for adhoc LLM calls and Pydantic
from pydantic import BaseModel
import aisuite as ai

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

research_agent = Agent(
    role="You are a helpful assistant that can answer questions about the web.",
    goal="Answer the user's question.",
    backstory="You have access to a vast knowledge base of information from the web.",
    tools=[
      WebsiteSearchTool(website=urls[0]),
      WebsiteSearchTool(website=urls[1]),
      WebsiteSearchTool(website=urls[2]),
    ],
    llm="gpt-4o-mini",
)

task = Task(
  description="Answer the following question: {question}",
  expected_output="A detailed and accurate answer to the user's question.",
  agent=research_agent,
)

crew = Crew(
    agents=[research_agent],
    tasks=[task],
)

/Users/joaomoura/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


# Creating State

In [4]:
class QAState(BaseModel):
  """
  State for the documentation flow
  """
  question: str = "What does Lilian Weng say about the types of agent memory?"
  improved_question: str = ""
  answer: str = ""

# Creating Flow

In [5]:
class QAFlow(Flow[QAState]):
  @start()
  def rewrite_question(self):
    print(f"# Rewriting question: {self.state.question}")
    client = ai.Client()
    messages = [
        {
          "role": "system",
          "content": f"""Look at the input and try to reason about the underlying semantic intent / meaning.
            Here is the initial question:
            -------
            {self.state.question}
            -------
            Formulate an improved question:"""
        }
    ]

    response = client.chat.completions.create(
        model="openai:gpt-4o-mini",
        messages=messages,
        temperature=0.3
    )

    print(response)

    improved_question = response.choices[0].message.content
    self.state.improved_question = improved_question

  @listen(rewrite_question)
  def answer_question(self):
    print(f"# Answering question: {self.state.improved_question}")
    result = crew.kickoff(inputs={'question': self.state.improved_question})
    self.state.answer = result.raw
    return result


# Plotting Flow

In [12]:
flow = QAFlow()
flow.plot()

# Display the flow visualization using HTML
from IPython.display import IFrame
IFrame(src='crewai_flow.html', width='100%', height=600)

Plot saved as crewai_flow.html


# Kicking off Flow

In [7]:
result = flow.kickoff()
print("=" * 10)
print(result)

# Rewriting question: What does Lilian Weng say about the types of agent memory?
ChatCompletion(id='chatcmpl-Aeo4gBp6YJNqtm6QW3RVqcSoIvcBo', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='What insights does Lilian Weng provide regarding the different types of agent memory in her work?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1734288970, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_6fc10e10eb', usage=CompletionUsage(completion_tokens=19, prompt_tokens=56, total_tokens=75, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
# Answering question: What insights does Lilian Weng provide regarding the different types of agent memory in her work?
In her work, Lilia

In [8]:
import time
final_time = time.time()
print(f"Total execution time: {final_time - initial_time:.2f} seconds")

Total execution time: 158.21 seconds
